In [3]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

df = pd.read_csv('D:/My WorkSpace/Summer-Internship/23-July-2024/flipkart.csv')

def label_rating(Rating):
    if Rating >= 4:
        return 2 # represents positive
    elif Rating == 3:
        return 1 # neutral
    else:
        return 0 # negative

df['labels'] = df['Rating'].apply(label_rating)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
data = pad_sequences(sequences, maxlen=200)

x_train, x_test, y_train, y_test = train_test_split(data, df['labels'], test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

def predict_user_input(text):
    sequences = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(sequences, maxlen=200)
    prediction = model.predict(padded_seq)
    label = prediction.argmax(axis=1)[0]
    if label == 2:
        return "Customer Satisfied"
    elif label == 1:
        return "Customer Partially Satisfied"
    else:
        return "Customer Not Satisfied"

root = tk.Tk()
root.title("Sentiment Analysis")

tk.Label(root, text="Enter your review:").pack(pady=10)
review_entry = tk.Entry(root, width=50)
review_entry.pack(pady=5)

predict_button = tk.Button(root, text="Predict Sentiment", command=lambda: result_label.config(text=f"Prediction: {predict_user_input(review_entry.get())}"))
predict_button.pack(pady=20)

result_label = tk.Label(root, text="Prediction will appear here.")
result_label.pack(pady=10)
root.mainloop()


Epoch 1/10


c:\Users\sachi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


47/47 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step - accuracy: 0.7906 - loss: 0.7852 - val_accuracy: 0.8266 - val_loss: 0.5668
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 192ms/step - accuracy: 0.8413 - loss: 0.5014 - val_accuracy: 0.8455 - val_loss: 0.4728
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 183ms/step - accuracy: 0.8924 - loss: 0.2998 - val_accuracy: 0.8672 - val_loss: 0.3811
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 185ms/step - accuracy: 0.9156 - loss: 0.2145 - val_accuracy: 0.8997 - val_loss: 0.3437
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 8s 168ms/step - accuracy: 0.9567 - loss: 0.1491 - val_accuracy: 0.8970 - val_loss: 0.3159
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 8s 165ms/step - accuracy: 0.9656 - loss: 0.1131 - val_accuracy: 0.9051 - val_loss: 0.3439
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 8s 168ms/step - accuracy: 0.9756 - loss: 0.0796 - val_accuracy: 0.9024 - val_loss: 0.3562
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 8s 166ms/step - accuracy: 0.9798 - loss: 0.0737 - val_accuracy: 0.9051 - val